In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

import wandb
from wandb.keras import WandbCallback

import os

2021-10-18 23:35:15.683665: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
import tensorflow as tf

os.environ['CUDA_VISIBLE_DEVICES'] = '0' # 초기화 할 GPU Number
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
sess = tf.compat.v1.Session(config=config)

2021-10-18 23:35:16.712055: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-10-18 23:35:16.713747: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-18 23:35:16.756189: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 23:35:16.756712: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce GTX 1660 SUPER computeCapability: 7.5
coreClock: 1.86GHz coreCount: 22 deviceMemorySize: 5.80GiB deviceMemoryBandwidt

In [3]:
IMG_SIZE = 224
BATCH_SIZE = 32

In [4]:
vgg_model = tf.keras.applications.VGG16(weights = 'imagenet',
                                        include_top = False,
                                        input_shape=(IMG_SIZE, IMG_SIZE, 3))
vgg_model.summary()

2021-10-18 23:35:17.147198: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-18 23:35:17.147370: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-18 23:35:17.147844: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:08:00.0 name: NVIDIA GeForce GTX 1660 SUPER computeCapability: 7.5
coreClock: 1.86GHz coreCount: 22 deviceMemorySize: 5.80GiB deviceMemoryBandwidth: 312.97GiB/s
2021-10-18 23:35:17.147868: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2021-10-18 23:35:17.147890: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-10-18 23:35:17.147902: I tensorflow/stream_executor/pla

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [5]:
model = tf.keras.Sequential()

for layer in vgg_model.layers[:-1]: # just exclude last layer from copying
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False

model.add(tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(tf.keras.layers.GlobalAveragePooling2D())
model.add(tf.keras.layers.Dense(2, activation='softmax'))

In [6]:
train_data_dir = '0310_dataset/train'
validation_data_dir = '0310_dataset/validation'
test_data_dir = '0310_dataset/test'

train_datagen = ImageDataGenerator(
                    rescale=1./255,
#                     rotation_range=30,
#                     width_shift_range=0.3,
#                     height_shift_range=0.3,
#                     fill_mode='nearest'
                    )

validation_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
train_data_dir,
target_size=(IMG_SIZE, IMG_SIZE),
batch_size=batch_size,
class_mode='categorical',
)

validation_generator = validation_datagen.flow_from_directory(
validation_data_dir,
target_size=(IMG_SIZE, IMG_SIZE),
batch_size=batch_size,
class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 2447 images belonging to 2 classes.
Found 612 images belonging to 2 classes.
Found 961 images belonging to 2 classes.


In [7]:
from tensorflow.keras.optimizers import Adam
from datetime import datetime

In [8]:
model.compile(loss='categorical_crossentropy',
             optimizer=tf.optimizers.Adam(),
             metrics=['accuracy']
             )

nb_train_samples = len(train_generator.classes)
nb_validation_samples = len(validation_generator.classes)

In [9]:
wandb.init(project='KICS_FALL', entity='yhkim')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: yhkim (use `wandb login --relogin` to force relogin)
/home/yongho/anaconda3/envs/tensor/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
2021-10-18 23:35:22.626841: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [10]:
config = wandb.config
config.epochs = 100

In [11]:
history = model.fit(train_generator,
                              steps_per_epoch=nb_train_samples//batch_size,
                              validation_data=validation_generator,
                              validation_steps=nb_validation_samples//batch_size,
                              epochs=config.epochs,
                    callbacks=[WandbCallback()])

2021-10-18 23:35:26.455483: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-18 23:35:26.472370: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3593330000 Hz


Epoch 1/100


2021-10-18 23:35:26.874512: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-10-18 23:35:27.177323: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7
2021-10-18 23:35:28.121172: W tensorflow/core/common_runtime/bfc_allocator.cc:248] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.46GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


76/76 [==============================] - 30s 363ms/step - loss: 0.7389 - accuracy: 0.7660 - val_loss: 0.2343 - val_accuracy: 0.9178
Epoch 2/100
76/76 [==============================] - 27s 353ms/step - loss: 0.2088 - accuracy: 0.9250 - val_loss: 0.1613 - val_accuracy: 0.9408
Epoch 3/100
76/76 [==============================] - 26s 347ms/step - loss: 0.1705 - accuracy: 0.9351 - val_loss: 0.1980 - val_accuracy: 0.9243
Epoch 4/100
76/76 [==============================] - 26s 345ms/step - loss: 0.1444 - accuracy: 0.9481 - val_loss: 0.1436 - val_accuracy: 0.9441
Epoch 5/100
76/76 [==============================] - 26s 346ms/step - loss: 0.1193 - accuracy: 0.9570 - val_loss: 0.1880 - val_accuracy: 0.9375
Epoch 6/100
76/76 [==============================] - 26s 346ms/step - loss: 0.1275 - accuracy: 0.9511 - val_loss: 0.1613 - val_accuracy: 0.9408
Epoch 7/100
76/76 [==============================] - 26s 346ms/step - loss: 0.0987 - accuracy: 0.9661 - val_loss: 0.1402 - val_accuracy: 0.9474
Epoc

In [ ]:
history = model.history

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
model.save('paper_data')

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import random
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix, accuracy_score
import sklearn

In [ ]:
model = load_model('./paper_data')
classes = os.listdir('./paper_dataset/train')

In [ ]:
img_rows, img_cols = 224, 224
batch_size = 64

test_data_dir = 'paper_dataset/test'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_rows, img_cols),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
test_generator.reset()
Y_pred = model.predict_generator(test_generator, verbose=1)
classes = test_generator.classes[test_generator.index_array]
y_pred = np.argmax(Y_pred, axis=-1)

In [ ]:
print(sum(y_pred==classes)/833)

## Scene Classification

In [ ]:
print('-- Confusion Matrix --')
print(confusion_matrix(test_generator.classes[test_generator.index_array], y_pred))

In [ ]:
class_names = ['not_pitching', 'pitching']

In [ ]:
print(classification_report(test_generator.classes[test_generator.index_array], y_pred, target_names=class_names))

## Object Detection

In [ ]:
od_true = np.zeros((833,), dtype=int)
od_predict = np.zeros((833,), dtype=int)

for x in os.listdir('paper_dataset/test/pitching'):
    od_true[int(x.replace('.','_').split('_')[1])] = 1

for x in os.listdir('od_result'):
    od_predict[int(x.replace('.','_').split('_')[1])] = 1

In [ ]:
print('-- Confusion Matrix --')
print(confusion_matrix(od_true, od_predict))

In [ ]:
print(classification_report(od_true, od_predict, target_names=class_names))

In [ ]:
accuracy_score(od_true, od_predict)

In [ ]:
scores = model.evaluate_generator(test_generator)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

In [ ]:
model.metrics_names

In [ ]:
# # random_img = random.sample(os.listdir('./paper_dataset/test/not_pitching/'), 1)[0]
# img = cv2.imread(f"./paper_dataset/test/pitching/frame_515.jpg")
# resize_img = cv2.resize(img.astype('float32') / 255, (224, 224))
# # print(random_img)

# output = model.predict(resize_img[np.newaxis])

# plt.title(f"True : not_pitching       Predict : {'not pitching' if np.argmax(output) == 0 else 'pitching'}")
# plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
# plt.savefig('Predict not_pitching scene', format='jpg')
# # cv2.imwrite('Predict pitching scene.jpg', img)
# # plt.imsave('Predict pitching scene.jpg', cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [ ]:
# outputs = model.predict_generator(test_generator, verbose=1)
# np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})
# print(test_generator.class_indices)
# print(outputs)

In [ ]:
# result = []
# for x in output:
#     result.append(np.argmax(x))

In [ ]:
# not_pitching = [x for x in result if x == 0]
# pitching = [x for x in result if x == 1]

# print(len(not_pitching))
# print(len(pitching))

In [ ]:
# outputs[213]

In [ ]:
# import shutil
# from glob import glob
# from sklearn.model_selection import train_test_split

In [ ]:
# src = '../0310_dataset'
# dst = '../0310_dataset/train'

### Train 폴더에 데이터 전부 옮기기

In [ ]:
# dir_list = os.listdir(src)
# dir_list.remove('train')
# dir_list.remove('validation')
# print(dir_list)
# cnt = 0
# for x in dir_list:
#     for y in os.listdir(f'{src}/{x}'):
#         if y.endswith('.jpg'):
#             print(y)
#             shutil.move(f'{src}/{x}/{y}', f'{dst}/{str(cnt).zfill(5)}.jpg')
#             shutil.move(f'{src}/{x}/{y.replace(".jpg", ".txt")}', f'{dst}/{str(cnt).zfill(5)}.txt')
#             cnt += 1
# print(cnt)

In [ ]:
# data = os.listdir(f'{src}/train')
# img_list = []

# for x in data:
#     if x.endswith('.jpg'):
#         img_list.append(x)


# print(len(img_list))
# # print(img_list)

# train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=42)

# print(len(train_img_list), len(val_img_list))

In [ ]:
# for x in val_img_list:
#     shutil.move(f'{src}/train/{x}', f'{src}/validation/{x}')